(tmux_custom-environment)=
# Создание настраиваемых рабочих окружений в tmux

Чаще всего разработчику при работе над проектом, приходится запускать целый набор различных инструментов и программ.
Например, веб-разработчику, скорее всего, потребуется командная оболочка, текстовый редактор, консоль базы данных и окно
для запуска автоматизированного набора тестов для разрабатываемого приложения.
Для всего этого необходимо множество окон и требуется вводить множество команд, чтобы все это запустить.

Было бы гораздо удобнее иметь возможность запускать все эти программы, каждую в своей панели или окне, в одном сеансе
tmux, используя всего одну команду.
Этого можно добиться, используя клиент-серверную модель tmux для создания пользовательских сценариев, которые создают
среды разработки, разделяя окна и автоматически запуская в них необходимые программы.
Сначала рассмотрим, как сделать это вручную, а затем рассмотрим более продвинутые автоматические инструменты.

## Создание пользовательских настроек с помощью команд tmux

Помимо создания новых сеансов, команда `tmux` имеет множество других параметров.
Например, можно разделить окна на панели, изменять макеты или даже запускать приложения в рамках сеанса.

Ключом к этому является опция `-t` или «target».
Если существует именованный сеанс tmux, к нему можно подключиться вот так:
```console
$ tmux attach -t [session_name]
```

Можно использовать этот указатель, чтобы применить команду tmux к нужному сеансу tmux.

Создадим новый сеанс tmux под названием «development»:
```console
$ tmux attach -s development
```

Отсоединимся от сеанса с помощью `PREFIX` `d`.

Можно разделить окно горизонтали, даже в неактивном сеансе tmux по, выполнив эту команду:
```console
$ tmux split-window -h -t development
```

Присоединитесь к сеансу еще раз, чтобы убедиться, что команда была выполнена:
```console
$ tmux attach -t development
```

На самом деле не нужно даже отключаться от сеанса tmux для отправки команд.
Можно просто открыть другой терминал и снова разделить окно, но на этот раз вертикально.
Откроем второе окно или вкладку терминала и введем следующую команду:
```console
$ tmux split-window -v -t development
```

Используя этот подход, можно легко настроить свое окружение.
Давайте рассмотрим эту концепцию, создав нашу собственную среду разработки.

## Сценарий конфигурации проекта

В разделе {ref}`«Изучение основ» <tmux_basic>` были описаны команды tmux `new-session` и `new-window`.
Давайте, используя эти и подобные команды, напишем простой скрипт который запустит новый сеанс tmux, создаст основное
окно с парой панелей и два дополнительных окна с одной панелью в каждом.
И напоследок, запустит отдельное приложение в каждой из панелей.

Начнем с создания нового скрипта под названием `development` в домашнем каталоге.
Сделаем этот скрипт исполняемым файлом, чтобы можно было запустить его, как и исполняемую программу в терминале.
Для этого выполним эти команды в терминале:
```console
$ touch ~/development
$ chmod +x ~/development
```

Пусть после запуска сеанса tmux переходит в каталог для проекта.
Назовем этот каталог `devproject`.
Но прежде чем перейти в этот каталог, лучше сначала его создать.

```console
$ mkdir ~/devproject
```

Теперь откроем скрипт `~/development` в текстовом редакторе и добавим команду, создающую новый сеанс tmux с именем "development":
Передадим пару дополнительных параметров при создании этого нового сеанса.
Во-первых, присвоим ему имя с помощью опции `-s`.
Во-вторых, назначим окну имя «editor».
И, наконец, немедленно отсоединяемся от этого нового сеанса с помощью опции `-d`.
```
# ~/development

tmux new-session -s development -n editor -d
```

Затем, используя команду tmux `send-keys`, измененим текущий каталог на каталог проекта:
```
# ~/development

tmux send-keys -t development 'cd ~/devproject' C-m
```
Последовательность `C-m` в конце строки имитирует последовательность возврата каретки, представленную как
{bdg-dark}`Ctrl`-{bdg-dark}`M`.
Так «заставит» tmux нажать клавишу {bdg-dark}`Enter` после ввода команды.

Используем тот же подход, чтобы открыть текстовый редактор Vim в этом окне:
```
# ~/development

tmux send-keys -t development 'vim' C-m
```

С помощью этих трех команд мы создали новый сеанс, перешли в каталог и открыли текстовый редактор, но наша среда еще не
настроена до конца.
Давайте разделим главное окно  пополам по горизонтали «editor», чтобы у нас было небольшое окно терминала внизу.
Сделаем это с помощью команды `split-window `:
```
# ~/development

`tmux split-window -v -t development
```

Можно было бы указать процент, используя что-то вроде
```
# ~/development

tmux split-window -v -p 10 -t development
```
но пока просто оставим команду `split-window` как есть, а затем выберем один из макетов tmux по умолчанию ---
`main-horizontal`:
```
# ~/development

tmux select-layout -t development main-horizontal
```